In [1]:

from pyphm.datasets.ims import ImsDataLoad
from pathlib import Path
import pandas as pd
import os
import numpy as np
import time
import datetime
import csv

%load_ext autoreload
%autoreload 2

In [2]:
root_dir = Path.cwd().parent
print(root_dir)
path_data_raw_folder = Path(root_dir / 'data' )
print(path_data_raw_folder)

/home/tim/Documents/PyPHM
/home/tim/Documents/PyPHM/data


In [3]:
ims = ImsDataLoad(path_data_raw_folder, 'ims', download=False)

path_1st_folder = ims.path_1st_folder
path_2nd_folder = ims.path_2nd_folder
path_3rd_folder = ims.path_3rd_folder

file_list = sorted(os.listdir(path_1st_folder))
file_name = file_list[0]

type(root) =  <class 'pathlib.PosixPath'>
/home/tim/Documents/PyPHM/data/ims


In [4]:
df = ims.load_run_as_df(1)

len file_list: 2156
Processing 2003.10.22.12.06.24, i=0 - run 1
Processing 2003.10.22.16.54.13, i=50 - run 1
Processing 2003.10.23.01.04.13, i=100 - run 1
Processing 2003.10.23.09.24.13, i=150 - run 1
Processing 2003.10.29.21.59.46, i=200 - run 1
Processing 2003.10.31.02.39.46, i=250 - run 1
Processing 2003.10.31.12.41.44, i=300 - run 1
Processing 2003.10.31.21.01.44, i=350 - run 1
Processing 2003.11.01.05.21.44, i=400 - run 1
Processing 2003.11.01.13.41.44, i=450 - run 1
Processing 2003.11.01.22.01.44, i=500 - run 1
Processing 2003.11.07.16.31.44, i=550 - run 1
Processing 2003.11.08.12.21.44, i=600 - run 1
Processing 2003.11.08.20.41.44, i=650 - run 1
Processing 2003.11.09.05.01.44, i=700 - run 1
Processing 2003.11.09.13.15.58, i=750 - run 1
Processing 2003.11.10.01.05.58, i=800 - run 1
Processing 2003.11.10.09.25.58, i=850 - run 1
Processing 2003.11.14.12.12.17, i=900 - run 1
Processing 2003.11.14.20.58.46, i=950 - run 1
Processing 2003.11.15.05.18.46, i=1000 - run 1
Processing 2003.

In [6]:
df.shape

(44154880, 12)

# Scratch

In [ ]:
def process_raw_csv(path_run_folder, file_name, sample_index, run_no, sample_freq=20480.0):

    signals_array = np.loadtxt(path_run_folder / file_name, delimiter="\t")

    id_list = [f'{run_no}_{sample_index}'] * len(signals_array)
    run_list = [run_no] * len(signals_array)
    file_list = [file_name] * len(signals_array)
    time_step_array = np.linspace(0.0, len(signals_array) / sample_freq, len(signals_array))

    return signals_array, id_list, run_list, file_list, time_step_array

In [ ]:
file_list = sorted(os.listdir(path_1st_folder))
file_name = file_list[0]

(
    signals_array, 
    id_list, 
    run_list, 
    file_list, 
    time_step_array) = process_raw_csv(path_1st_folder, file_name, 0, 1, sample_freq=20480.0)

In [ ]:
a = np.array(range(0,4))
b = np.array(range(4,8))
c = np.array(range(8,12))
d = np.hstack([a, b, c])
d

In [ ]:
df = ims.load_run_as_df(1)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
run.extend(run)


In [ ]:
len(run)

In [ ]:
a = [signals, signals]
a = np.concatenate(a, axis=0)
a.shape

In [ ]:
df = pd.DataFrame(a, columns=ims.col_1st_names)
df['run'] = run
df.tail()

In [ ]:
run_no = 1
sample_index = 0
id_list = [f'{run_no}_{sample_index}']*10

id_list

In [ ]:
a = np.loadtxt(path_1st_folder / file_name, delimiter="\t")

col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
df = pd.read_csv(path_1st_folder / file_name, sep='\t', names=col_names)
df.head()

In [ ]:
a[1]

In [ ]:
df = ims.load_run_as_dataframe(run_no=1)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
path_1st_folder = ims.path_1st_folder
path_2nd_folder = ims.path_2nd_folder
path_3rd_folder = ims.path_3rd_folder

In [ ]:
file_list = sorted(os.listdir(path_3rd_folder))
file_list[0]

In [ ]:
file_name = file_list[0]
print(file_name)

col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
df = pd.read_csv(path_3rd_folder / file_name, sep='\t', names=col_names)
df.head()

In [ ]:
# columns to add
# ['id', 'run', 'start_time_unix', 'date_time', 'unix_time', 'file_name']

df['id'] = "1_" + file_name
df['run'] = 1
df.head()

In [ ]:
def process_raw_csv_ims(path_run_folder, file_name, sample_index, run_no, col_names, sample_freq=20480.0):

    df = pd.read_csv(path_run_folder / file_name, sep='\t', names=col_names)

    df['id'] = "1_" + str(sample_index)
    df['run'] = run_no
    df['file'] = file_name
    df['time'] = np.linspace(0.0, len(df)/sample_freq, len(df))

    return df[['id', 'file', 'run', 'time',] + col_names]

In [ ]:
df1 = process_raw_csv_ims(path_3rd_folder, file_name, 0, 3, col_names, sample_freq=20480.0)
df1.head()

In [ ]:
def create_xy_dataframe():

In [ ]:
df1['time'][2]

In [ ]:
# convert start_time to unix timestamp
start_time_unix = time.mktime(
    datetime.datetime.strptime(file_name, "%Y.%m.%d.%H.%M.%S").timetuple()
)
start_time

In [ ]:
date_nice_format = datetime.datetime.fromtimestamp(start_time).strftime(
    "%Y-%m-%d %H:%M:%S"
)  # reformat date
date_nice_format


In [ ]:
date_list = sorted(os.listdir(path_1st_folder))

In [ ]:
date_list[:10]